# Progetto MCS

Per la gestione della struttura dati e le operazioni elementari fra matrici è richiesto di partire da una libreria open-source, come Eigen, Armadillo, blas/lapack. Oppure, qualora il linguaggio di programmazione lo permetta utilizzare vettori e matrici già implementate al suo interno.


## Data Import

In [ ]:
import numpy as np
from scipy.io import mmread
from scipy.sparse import csr_matrix, csc_matrix, tril
from scipy.sparse.linalg import spsolve_triangular

In [ ]:
data = {
    "spa1" : 0,
    "spa2" : 0,
    "vem1" : 0,
    "vem2" : 0}
for x in data:
    data[x] = {
        "A" : mmread("data/" + x + ".mtx"),
        "x" : 0,
        "b" : 0,
    }
    data[x]["A"] = csc_matrix(data[x]["A"])
    data[x]["x"] = np.array([1.0]*data[x]["A"].get_shape()[0])
    data[x]["b"] = np.array(data[x]["A"].dot(data[x]["x"]))
tols = [10**(-4), 10**(-6), 10**(-8), 10**(-10)]

# TODO
### - Cosa che resituisca i tempi delle singole esecuzioni
### - Errore assoluto (se serve)
### - Grafici
### - Metodo preliminare per i metodi che lo hanno (es. jacobi che dice subito se la matrice è risolvibile o meno )
### - Fare file di python eseguibile
### - Sistemare e capire come raggruppare il codice

In [ ]:
#jacobi
from jacoby_mcs import Jacoby
ja = Jacoby()
files_jacobi = {}
for file in data:
    jacobi_results=[]
    for tol in tols:
        jacobi_results.append(ja.execute(data[file]["A"], data[file]["b"], data[file]["x"], tol))
    files_jacobi[file] = jacobi_results

In [ ]:
files_jacobi

In [ ]:
#gradiente_coniugato
from gradiente_coniugato import GradC
gc = GradC()
files_gradC = {}
for file in data:
    gradC_results=[]
    for tol in tols:
        gradC_results.append(gc.execute(data[file]["A"], data[file]["b"], data[file]["x"], tol))
    files_gradC[file] = gradC_results

In [ ]:
files_gradC

Condizione di arresto: $\frac{||Ax^{(k)}-b||}{||b||}<tol$

Reminder: 

1.   **Iniziare le iterazioni con il vettore x nullo**
2.   **tol = [$10^{-4}, 10^{-6}, 10^{-8}, 10^{-10}]$**
3.   **Dichiarare di non essere giunti a convergenza se k > maxiter dove maxiter lo scegliamo (>= 20000)**




## Metodo di Jacobi
https://en.wikipedia.org/wiki/Jacobi_method

In [ ]:
niter = 0
new_vector = np.zeros(spa1.get_shape()[0])
inverted_p_matrix = inv(np.diagflat(spa1.diagonal()))
tol = 1e-4
while True:
    residual = spa1b - spa1.dot(new_vector)
    new_vector = new_vector + (inverted_p_matrix.dot(residual))
    niter = niter +1
    if(np.linalg.norm(residual, ord=2)/np.linalg.norm(spa1b, ord=2) < tol or niter >= 20000):
      break;
print(niter)
print(new_vector)

In [ ]:
#jacobi secondo slide del prof
niter = 0
new_vector = np.asarray([0]*len(spa1x))
inverted_p_matrix = 1/spa1.diagonal()
tol = 10**(-4)
residual = spa1b - spa1.dot(new_vector)
while np.linalg.norm(residual)/np.linalg.norm(spa1b) >= tol and niter <= 20000:
    new_vector = new_vector + (inverted_p_matrix * (residual))
    residual = spa1b - spa1.dot(new_vector)
    niter = niter +1
    print(np.linalg.norm(residual)/np.linalg.norm(spa1b))

### Errore relativo - numero iterazioni - tempo di calcolo

In [ ]:
#numero iterazioni
niter

In [ ]:
#errore relativo
(np.linalg.norm(np.subtract(new_vector, spa1x))/np.linalg.norm(spa1x))*100

## Metodo di Gauß-Seidel

In [ ]:
## Creating Working Variables
mtxA = data["vem2"]["A"].copy().tocsr()
mtxP = tril(mtxA, format="csr")
max_iter = 20000
vectB = data["vem2"]["b"].copy()
tol = tols[3]
k = 0
vectX = np.zeros(mtxA.shape[0])
residual = vectB - mtxA.dot(vectX)

while np.linalg.norm(residual)/np.linalg.norm(vectB) >= tol and k < max_iter:
    k += 1
    vectX = vectX + spsolve_triangular(mtxP,residual, lower = True)
    residual = vectB - mtxA.dot(vectX)
    
    if k%100 == 0:
        print(k, end = "\t")
        if k%1000 == 0:
            print()
            print(np.linalg.norm(residual))
            print(np.linalg.norm(residual)/np.linalg.norm(vectB))
print(k)
print(np.linalg.norm(residual))
print(np.linalg.norm(residual)/np.linalg.norm(vectB))
print(vectX)

In [ ]:
import gauss_seidel as gs

for el in data:
    for tol in tols:
        gs.solve(mtxA=data[el]["A"], vectB=data[el]["b"], tol=tol, maxIter=20000)

## Metodo del Gradiente


## Metodo del Gradiente coniugato

- Un vettore ottimale rispetto a una direzione d se d*r(k)=0
- x(k+1) è ottimale rispetto a r(k+1)
- x(k+1) = x(k) + a(k)d(k)
- a(k) = ( d(k)^t * r(k) ) / ( d(k)^t * Ad(k) )
- d(k+1) = r(k+1) - b(k)*d(k)
- b(k) = ( d(k)^t * Ar(k+1) ) / ( d(k)^t * Ad(k) )



In [ ]:
#gradiente coniugato
#capire se in ak e bk bisogna calcolare la derivata di dir
niter = 0
new_vector = np.asarray([0]*len(spa1x))
tol = 10**(-4)
residual = spa1b - spa1.dot(new_vector)
dir = residual.copy()
while np.linalg.norm(residual)/np.linalg.norm(spa1b) >= tol and niter <= 20000:
    #r = spa1b - spa1.dot(new_vector)
    y = spa1.dot(dir)
    z = spa1.dot(residual)
    ak = (np.dot(dir, residual)) / (np.dot(dir, y))
    new_vector = new_vector + (ak * dir)
    residual = spa1b - spa1.dot(new_vector)
    w = spa1.dot(residual)
    bk = (np.dot(dir, w)) / (np.dot(dir, y))
    dir = residual - (bk*dir)

    residual = spa1b - spa1.dot(new_vector)
    niter = niter + 1

In [ ]:
niter

In [ ]:
np.linalg.norm(np.subtract(new_vector, spa1x))/np.linalg.norm(spa1b)